# scRFE Starter CodeV5

In [2]:
# 2020-05-13

In [3]:
# Imports 
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

In [30]:
# read in data and NORMALIZE
adata = read_h5ad('/Users/madelinepark/Downloads/Kidney_facs.h5ad')
sc.pp.normalize_total(adata, target_sum = 1e6, inplace = True)

In [31]:
# goal: get labels on a per class basis that will go into randomForest function for y
# THE ONE I HAVE BEEN USING THAT WORKS 
def getLabels (dataMatrix, classOfInterest): 
    labelsDict = {}
    for label in np.unique(adata.obs[classOfInterest]):
        lists = []        
        for obs in adata.obs[classOfInterest]:
            if obs == label: 
                lists.append('A')
            else:
                lists.append('B')
        labelsDict[label] = lists #this is usually in line w if and else    
    return labelsDict

In [57]:
def makeOneForest (dataMatrix, classOfInterest, labelOfInterest, nEstimators = 10, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv = 3): 
    
    print('makeOneForest' + labelOfInterest)
    labelsDict = getLabels(dataMatrix, classOfInterest) 

    feat_labels = dataMatrix.var_names #this is equivalent of the genes
    X = dataMatrix.X
    y = labelsDict[labelOfInterest]

    clf = RandomForestClassifier(n_estimators = nEstimators, random_state = randomState, 
                                 n_jobs = nJobs, oob_score = oobScore)
    selector = RFECV(clf, step = Step, cv = Cv)
    
    print('training...')
    clf.fit(X, y)
    selector.fit(X, y)
    feature_selected = feat_labels[selector.support_] 

    return feature_selected, selector.estimator_.feature_importances_ 

In [92]:
MakeOneForest = makeOneForest (dataMatrix = adata, classOfInterest = 'age', 
                               labelOfInterest ='24m',
                       nEstimators = 10, randomState = 0,  
                  nJobs =-1, oobScore= True, Step=0.2 , Cv=3)
        
genes = MakeOneForest[0]
ginis = MakeOneForest[1]

makeOneForest24m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

In [95]:
def resultWrite (classOfInterest, labelOfInterest, results_df,
                feature_selected, 
                 feature_gini):

#add genes and ginis as arguments in resultWrite
    print ('result writing')
#     somewhere I need to say what feature selected and feature gini are 
    resaux = pd.DataFrame(columns = [labelOfInterest, labelOfInterest+'_gini'])
    resaux[labelOfInterest] = feature_selected
    resaux[labelOfInterest + '_gini'] = feature_gini
#     sort df by gini column
    results_df = pd.concat([results_df,resaux],axis=1)
    results_df.sort_values(by='')
    sort_values(by = [labelOfInterest + '_gini'])

    return results_df 

In [76]:
# set genes, ginis from makeOneForest, then feed those and data frame into 
# result write

In [ ]:
def foo():
    return("a", "b", "c", "d")

hi, hi1, hi2, hi3 = foo()

In [80]:
# try at night bc bored
def scRFE(dataMatrix, classOfInterest,  nEstimators = 10, randomState = 0,  
                  nJobs = -1, oobScore = True, Step = 0.2, Cv =3):
    
    results_df = pd.DataFrame() 
    
    for labelOfInterest in np.unique(dataMatrix.obs[classOfInterest]):    
        print( 'scRFE' + labelOfInterest)
        
        MakeOneForest = makeOneForest (dataMatrix, classOfInterest, labelOfInterest,
                       nEstimators, randomState,  
                  nJobs, oobScore, Step , Cv)
        
        genes = MakeOneForest[0]
        ginis = MakeOneForest[1]

        resultWrite (dataMatrix, classOfInterest, labelOfInterest, 
                     results_df, feature_selected = genes, feature_gini = ginis)
        
    return resultWrite

In [81]:
# def scRFE(dataMatrix, classOfInterest,  nEstimators = 10, randomState = 0,  
#                   nJobs = -1, oobScore = True, Step = 0.2, Cv =3):
    
#     results_df = pd.DataFrame() 
    
#     for label in np.unique(dataMatrix.obs[classOfInterest]):    
#         print( 'scRFE' + label)
#         makeOneForest (dataMatrix, classOfInterest, labelOfInterest, 
#                        nEstimators = 10, randomState = 0,  
#                   nJobs = -1, oobScore = True, Step = 0.2, Cv = 3): 
        
#         MakeOneForest = makeOneForest(dataMatrix = dataMatrix, 
#                                       classOfInterest = classOfInterest, 
#                       labelOfInterest = label, nEstimators = nEstimators, 
#                       randomState = randomState,  nJobs = nJobs, 
#                       oobScore = oobScore, Step = Step, Cv = Cv)

#         MakeOneForest = makeOneForest(dataMatrix = dataMatrix, 
#                                       classOfInterest = classOfInterest, 
#                       labelOfInterest = label)
    
#         genes = MakeOneForest[0]
#         ginis = MakeOneForest[1] #this can be sped up

#         resultWrite (dataMatrix, classOfInterest, results_df, labelOfInterest = label)
        
#     return resultWrite

In [82]:
try1 = scRFE (dataMatrix = adata, classOfInterest = 'age')

scRFE24m
makeOneForest24m
training...


/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/madelinepark/anaconda3/envs/scanpytest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/Users/madelinep

TypeError: resultWrite() got multiple values for argument 'feature_selected'

In [27]:
try1

,3m,3m_gini
0,0610007L01Rik,0.000000
1,0610007N19Rik,0.000000
2,0610007P08Rik,0.000000
3,0610007P14Rik,0.000000
4,0610007P22Rik,0.000000
5,0610008F07Rik,0.000000
6,0610009B14Rik,0.000000
7,0610009B22Rik,0.000000
8,0610009D07Rik,0.000000
9,0610009L18Rik,0.000000
